In [1]:
!nvidia-smi

Thu Jan  2 20:44:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:02:00.0 Off |                  N/A |
| 31%   48C    P8             22W /  350W |    5799MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
torch.cuda.empty_cache()  # Free up unutilized memory

In [3]:
from src.segmentation.framework_handlers import ultralytics_handler
ultralytics_handler.validate_version_and_gpu()

Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24253MiB)
Setup complete ✅ (12 CPUs, 125.2 GB RAM, 15.7/336.4 GB disk)


### Prepare Dataset
In this section, the dataset along with YOLO annotations is processed to conform to the input structure required by YOLO. The YOLO format necessitates organizing the dataset into two main directories, `images` and `labels`. Each of these directories is further divided into two subdirectories, `train` and `val`, for training and validation datasets respectively. This structure facilitates YOLO's training and validation processes.
WE can process the data by two options:
- Single SegmentsAI dataset
- Two SegmentsAI datasets, one for train and the other for test

Option 1: prepare data using a single dataset by splitting to train and test

### Ultralytics Settings

In here we can modify the settings of the yolo runs. 

In [7]:
from ultralytics import settings
import config

# Update a setting
yolo_runs_dir = "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs"
settings.update(
    {
        'runs_dir': yolo_runs_dir,
        'weights_dir': f"{yolo_runs_dir}/weights",
        'datasets_dir': f"{yolo_runs_dir}/datasets",
    }
)

# View all settings
print(settings)

JSONDict("/home/etaylor/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/datasets",
  "weights_dir": "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/weights",
  "runs_dir": "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs",
  "uuid": "01de958df926b4e70f7bb00e8bdc993b48b07bc8d023fc13b26874ee84610e6f",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}


In [5]:
# import os

# # in case dataset already converted from coco to yolo format use manual path
# yaml_file_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/yaml_files/yolo_dataset_config.yaml"

# # currently available models for train and hparams fine tune
# ultralytics_checkpoint_paths = "/home/etaylor/code_projects/thesis/checkpoints/ultralytics"

# available_models_checkpoints = {
#     "yolov5": "yolov5xu.pt",
#     "yolov8": "yolov8x.pt",
#     "yolov9": "yolov9c.pt",
#     "rt_detr": "rtdetr-x.pt",
#     "yolov10": "yolov10n.pt",
# }
# model_checkpoint_path = os.path.join(ultralytics_checkpoint_paths, available_models_checkpoints["yolov10"])

Here we can select between using the yolo models or the RT_DETR models

In [10]:
from ultralytics import YOLO

model = YOLO('yolov8m-seg.pt')

yaml_file_path = "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/yaml/stigmas_dataset_yolo.yaml"

In [9]:
yaml_file_path = "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/yaml/stigmas_dataset_yolo.yaml"

If using the RT DETR models use the code below

In [ ]:
# from ultralytics import RTDETR

# model_checkpoint_path = os.path.join(ultralytics_checkpoint_paths, available_models_checkpoints["rt_detr"])
# model = RTDETR(model_checkpoint_path)

In [11]:
# regular train of the model
results = model.train(data=yaml_file_path, epochs=50, imgsz=640)

Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24253MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/yaml/stigmas_dataset_yolo.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_dataset/yolo_formatted/labels/train... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<00:00, 310.35it/s]

train: New cache created: /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_dataset/yolo_formatted/labels/train.cache



/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_dataset/yolo_formatted/labels/val... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<00:00, 260.84it/s]

val: New cache created: /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_dataset/yolo_formatted/labels/val.cache


...

      50/50      8.59G     0.8551      1.658     0.6116      1.134         42        640: 100%|██████████| 6/6 [00:01<00:00,  3.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.42it/s]

                   all         25        190       0.61      0.621      0.616      0.345       0.59      0.474      0.448      0.137



50 epochs completed in 0.057 hours.
Optimizer stripped from /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/segment/train2/weights/last.pt, 54.8MB
Optimizer stripped from /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/segment/train2/weights/best.pt, 54.8MB

Validating /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/segment/train2/weights/best.pt...
Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24253MiB)
YOLOv8m-seg summary (fused): 245 layers, 27,222,963 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.74it/s]


                   all         25        190      0.575      0.626       0.59      0.334      0.622      0.453      0.461      0.152
Speed: 0.1ms preprocess, 3.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/segment/train2


In [12]:
valid_results = model.val()
print(valid_results)

Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24253MiB)
YOLOv8m-seg summary (fused): 245 layers, 27,222,963 parameters, 0 gradients, 110.0 GFLOPs


val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_dataset/yolo_formatted/labels/val.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.26s/it]


                   all         25        190      0.577       0.64      0.591      0.336      0.615      0.447      0.457      0.151
Speed: 0.1ms preprocess, 54.5ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/runs/segment/train22
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4a285c24c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    

In [ ]:
from IPython.display import Image as show_image
show_image(filename="/sise/home/etaylor/runs/segment/train22/val_batch0_pred.jpg")

### Perform Infrence with the model

In [8]:
import cv2

image = cv2.imread("/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train/v0.1/IMG_2198_p12.png")

# Perform inference
results = model.predict(image)


0: 640x640 (no detections), 19.5ms
Speed: 12.6ms preprocess, 19.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
